In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import geopy.distance
import multiprocessing
from multiprocessing import Pool
from sklearn import linear_model
from sklearn import metrics

In [3]:
from tqdm import tqdm
tqdm.pandas()

S:\Anaconda\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
train = pd.read_csv("train.csv" , date_parser=True)
train.head().T

,0,1,2,3,4
id,id2875421,id2377394,id3858529,id3504673,id2181028
vendor_id,2,1,2,2,2
pickup_datetime,2016-03-14 17:24:55,2016-06-12 00:43:35,2016-01-19 11:35:24,2016-04-06 19:32:31,2016-03-26 13:30:55
dropoff_datetime,2016-03-14 17:32:30,2016-06-12 00:54:38,2016-01-19 12:10:48,2016-04-06 19:39:40,2016-03-26 13:38:10
passenger_count,1,1,1,1,1
pickup_longitude,-73.9822,-73.9804,-73.979,-74.01,-73.9731
pickup_latitude,40.7679,40.7386,40.7639,40.72,40.7932
dropoff_longitude,-73.9646,-73.9995,-74.0053,-74.0123,-73.9729
dropoff_latitude,40.7656,40.7312,40.7101,40.7067,40.7825
store_and_fwd_flag,N,N,N,N,N


In [5]:
test = pd.read_csv("test.csv", date_parser=True)
test.head().T

,0,1,2,3,4
id,id3004672,id3505355,id1217141,id2150126,id1598245
vendor_id,1,1,1,2,1
pickup_datetime,2016-06-30 23:59:58,2016-06-30 23:59:53,2016-06-30 23:59:47,2016-06-30 23:59:41,2016-06-30 23:59:33
passenger_count,1,1,1,1,1
pickup_longitude,-73.9881,-73.9642,-73.9974,-73.9561,-73.9702
pickup_latitude,40.732,40.68,40.7376,40.7719,40.7615
dropoff_longitude,-73.9902,-73.9598,-73.9862,-73.9864,-73.9615
dropoff_latitude,40.7567,40.6554,40.7295,40.7305,40.7559
store_and_fwd_flag,N,N,N,N,N


In [6]:
print("Train Shape: " , train.shape)
print("Test Shape: " , test.shape)

Train Shape:  (1458644, 11)
Test Shape:  (625134, 9)


In [7]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
vendor_id,1458644.0,1.534950,0.498777,1.000000,1.000000,2.000000,2.000000,2.000000e+00
passenger_count,1458644.0,1.664530,1.314242,0.000000,1.000000,1.000000,2.000000,9.000000e+00
pickup_longitude,1458644.0,-73.973486,0.070902,-121.933342,-73.991867,-73.981743,-73.967331,-6.133553e+01
pickup_latitude,1458644.0,40.750921,0.032881,34.359695,40.737347,40.754101,40.768360,5.188108e+01
dropoff_longitude,1458644.0,-73.973416,0.070643,-121.933304,-73.991325,-73.979752,-73.963013,-6.133553e+01
dropoff_latitude,1458644.0,40.751800,0.035891,32.181141,40.735885,40.754524,40.769810,4.392103e+01
trip_duration,1458644.0,959.492273,5237.431724,1.000000,397.000000,662.000000,1075.000000,3.526282e+06


In [ ]:
train.sample(5)

### This tells us that there are no missing values in the dataset

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Calculate distance using co-ordinates using geopy

In [9]:
import time
train_temp = pd.read_csv("train.csv")
geo_data = []
for plong,plat,dlong,dlat in zip(train_temp.pickup_longitude,
                                 train_temp.pickup_latitude,
                                 train_temp.dropoff_longitude,
                                 train_temp.dropoff_latitude):
    
    if (plat<-90 or plat>90) or (dlat<-90 or dlat>90):
        print("yes", plat, dlat)
    
    point = (plat,plong,dlat,dlong)
    geo_data.append(point)

In [10]:
def cal_km(geo_points):
    coord_1 = (geo_points[0],geo_points[1])
    coord_2 = (geo_points[2],geo_points[3])
    try:
        d = geopy.distance.geodesic(coord_1, coord_2).km
        return d
    except Exception as e:
        print(geo_points)
        return np.NaN

In [ ]:
print("CPU count: ", multiprocessing.cpu_count())
start_time = time.time()
with Pool(6) as p:
    distance = p.map(cal_km, geo_data)

print("{} seconds".format(time.time() - start_time))

CPU count:  8


In [ ]:
K = 5
Centroids = (X.sample(n=K))
plt.scatter(X['pickup_longitude'], X['pickup_latitude'], c='red')
plt.title('Pickup Location Scatter plot')
plt.xlabel('pickup_longitude')
plt.ylabel('pickup_latitude')
plt.scatter(Centroids["pickup_longitude"],Centroids["pickup_latitude"],c='green')
plt.show()

In [ ]:
X = train[['pickup_longitude','pickup_latitude']]
K = 100
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(type(X))
X = scaler.fit_transform(X)
print(type(X))
## X is numpy array after sacling, lets convert it back to a Dataframe
X = pd.DataFrame(X, columns=['pickup_longitude', 'pickup_latitude'])
type(X)
X

In [ ]:
X = train[['dropoff_longitude','dropoff_latitude']]
K = 100
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(type(X))
X = scaler.fit_transform(X)
print(type(X))
## X is numpy array after sacling, lets convert it back to a Dataframe
X = pd.DataFrame(X, columns=['dropoff_longitude', 'dropoff_latitude'])
type(X)
X

In [ ]:
Centroids = (X.sample(n=K))
plt.scatter(X['dropoff_longitude'], X['dropoff_latitude'], c='red')
plt.title('Drop Off Location Scatter plot')
plt.xlabel('dropoff_longitude')
plt.ylabel('dropoff_latitude')
plt.scatter(Centroids["dropoff_longitude"],Centroids["dropoff_latitude"],c='green')
plt.show()

In [ ]:
x = train['dropoff_longitude']
y = train['dropoff_latitude']
plt.scatter(x, y, c='blue')

x = train['pickup_longitude']
y = train['pickup_latitude']

plt.scatter(x, y, c='red')

plt.show()

## Scaling the training dataset

In [ ]:
train.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']] = scaler.fit_transform(train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']])
train.head()

In [ ]:
# fitting multiple k-means algorithms and storing the values in an empty list
SSE = []
from sklearn.cluster import KMeans
def Calculate_SSE():
    for cluster in range(1,30):
        kmeans = KMeans(n_jobs = -1, n_clusters = cluster, init='k-means++')
        kmeans.fit(train[['pickup_longitude','pickup_latitude']])
        SSE.append(kmeans.inertia_)
        
Calculate_SSE()
print(SSE)

In [ ]:
# SSE
# with Pool(8) as p:
#     SSE = p.map(Calculate_SSE , list(range(1,30)))
print("CPU count: ", multiprocessing.cpu_count())

In [ ]:
 # converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,30), 'SSE':SSE})

plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

### Based on the elbow method we select the value of k = 6

## Create a cluster for pickup co-ordinates

In [ ]:
kmeans = KMeans(n_jobs = -1, n_clusters = 6, init='k-means++')
kmeans.fit(train[['pickup_longitude','pickup_latitude']])
pred = kmeans.predict(train[['pickup_longitude','pickup_latitude']])
pred

In [ ]:
frame = pd.DataFrame()
# train['pickup_cluster'] = pred
train['pickup_cluster'] = pred
train['pickup_cluster'].value_counts()

## Do the same for dropoff co-ordintes

In [ ]:
kmeans = KMeans(n_jobs = -1, n_clusters = 6, init='k-means++')
kmeans.fit(train[['dropoff_longitude','dropoff_latitude']])
pred = kmeans.predict(train[['dropoff_longitude','dropoff_latitude']])
pred

In [ ]:
frame = pd.DataFrame()
# train['pickup_cluster'] = pred
train['dropoff_cluster'] = pred
train['dropoff_cluster'].value_counts()

## Lets analyze the date feauture of train and test sets

In [ ]:
print("Minimum Date: ",train['pickup_datetime'].min())
print("Maximum Date: ", train['pickup_datetime'].max())

In [ ]:
print("Minimum Date: ",test['pickup_datetime'].min())
print("Maximum Date: ", test['pickup_datetime'].max())

### Insight from above data: Its a six month period only, so year data feature has no relevance

In [ ]:
df = pd.DataFrame()
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
train.head(5)
train['pickup_datetime_month'] = train['pickup_datetime'].dt.month
train['pickup_datetime_dayofweek'] = train['pickup_datetime'].dt.dayofweek
train['pickup_datetime_hour'] = train['pickup_datetime'].dt.hour
train['pickup_datetime_minute'] = train['pickup_datetime'].dt.minute
train.head(5)

In [ ]:
train['pickup_datetime_dayofweek'].value_counts(normalize =True)*100

In [ ]:
train.pickup_datetime_hour.plot.hist(bins=24, ec="black")

In [ ]:
train.pickup_datetime_minute.plot.hist(bins=24, ec="black")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(train[['pickup_datetime_hour']]).toarray())
# merge with main df bridge_df on key values
enc_df

In [ ]:
train.head()

In [ ]:
import time
train_temp = pd.read_csv("train.csv")
geo_data = []
for plong,plat,dlong,dlat in zip(train_temp.pickup_longitude,
                                 train_temp.pickup_latitude,
                                 train_temp.dropoff_longitude,
                                 train_temp.dropoff_latitude):
    
    if (plat<-90 or plat>90) or (dlat<-90 or dlat>90):
        print("yes", plat, dlat)
    
    point = (plat,plong,dlat,dlong)
    geo_data.append(point)

def cal_km(geo_points):
    coord_1 = (geo_points[0],geo_points[1])
    coord_2 = (geo_points[2],geo_points[3])
    try:
        d = geopy.distance.geodesic(coord_1, coord_2).km
        return d
    except Exception as e:
        print(geo_points)
        return np.NaN
    
print("CPU count: ", multiprocessing.cpu_count())
start_time = time.time()

with Pool(6) as p:
    distance = p.map(cal_km, geo_data)

print("{} seconds".format(time.time() - start_time))